# 리뷰 기반 게임 검색

In [2]:
import os
import time
import pickle
import joblib
import sys

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer #CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk

from nltk.corpus import stopwords
from collections import Counter

In [7]:
#Dataframe

#open 피클 
with open("preprocessing/gmae_pkl/game_df_b120.pkl","rb") as fr:
    game_df = pickle.load(fr)


In [14]:
game_df_b50

,platform,title,publisher,0,1,2,3,4,5,6,...,90,91,92,93,94,95,96,97,98,99
0,DS,Ghost Trick: Phantom Detective,Capcom,"{'score': 10, 'review': 'Easily one of the bes...","{'score': 9, 'review': 'Hands down, one of the...","{'score': 10, 'review': 'Innovative and addict...","{'score': 10, 'review': 'Phenomenal. This game...","{'score': 10, 'review': 'I picked up this game...","{'score': 10, 'review': 'This game is perfecti...","{'score': 6, 'review': 'Sadly, I didn't enjoy ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PSP,Z.H.P. Unlosing Ranger vs Darkdeath Evilman,NIS America,"{'score': 8, 'review': 'Probably the best rogu...","{'score': 8, 'review': 'If you aren't familiar...","{'score': 10, 'review': 'NIS has always writte...","{'score': 9, 'review': 'It is really amazing t...","{'score': 10, 'review': 'Truly a unique experi...","{'score': 10, 'review': 'This game's name. 'Nu...","{'score': 1, 'review': 'It's one thing to make...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Xbox Series X,MADiSON,Perp Games,"{'score': 1, 'review': 'It's a very short game...","{'score': 2, 'review': 'The game is very poorl...","{'score': 10, 'review': 'Best horror experienc...","{'score': 10, 'review': 'An Argentine game tha...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PC,Final Fantasy XIV: Endwalker,Square Enix,"{'score': 10, 'review': 'Endwalker solidifies ...","{'score': 10, 'review': 'Square Enix somehow f...","{'score': 10, 'review': 'The best story, full ...","{'score': 10, 'review': 'Issa masterpiece. For...","{'score': 10, 'review': 'It seemed impossible,...","{'score': 10, 'review': 'What are temporary se...","{'score': 10, 'review': 'The story is even bet...",...,"{'score': 10, 'review': 'The queue are long, b...","{'score': 10, 'review': 'A great finale to the...","{'score': 10, 'review': 'This is the best expa...","{'score': 10, 'review': 'This game, this game ...","{'score': 10, 'review': 'Moving scenes, nice a...","{'score': 10, 'review': 'I like the story of t...","{'score': 10, 'review': 'They did what many of...","{'score': 10, 'review': 'For me, Endwalker is ...","{'score': 10, 'review': 'I... was afraid when ...","{'score': 10, 'review': 'One of the best JRPG/..."
4,PlayStation 3,Vanquish,Sega,"{'score': 6, 'review': 'The controls are great...","{'score': 7, 'review': 'Vanquish is a game tha...","{'score': 7, 'review': 'Repetitive and over th...","{'score': 10, 'review': 'This game has a sligh...","{'score': 10, 'review': 'I don't understand wh...","{'score': 9, 'review': 'I'm in love with Vanqu...","{'score': 9, 'review': 'It may not have had th...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PlayStation,Resident Evil,Capcom,"{'score': 9, 'review': 'Every room had its own...","{'score': 9, 'review': '1996. The year I was b...","{'score': 10, 'review': 'Heart-pumping classic...","{'score': 9, 'review': 'At the time when this ...","{'score': 10, 'review': 'This is where residen...","{'score': 10, 'review': 'Where do we start off...","{'score': 10, 'review': 'And so it begins! Mus...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,PlayStation 2,Resident Evil 4,Capcom,"{'score': 10, 'review': 'This game is the best...","{'score': 10, 'review': '-I'm on the side of t...","{'score': 10, 'review': 'I've played and past ...","{'score': 10, 'review': 'Best horror game! Bes...","{'score': 10, 'review': 'i swear to god. this ...","{'score': 10, 'review': 'Fun, exciting and eas...","{'score': 10, 'review': 'Best survivor horror ...",...,"{'score': 9, 'review': 'I'm not a fan of survi...","{'score': 9, 'review': 'The last/best atmosphe...","{'score': 10, 'review': 'Perrrrrrrrrrrrrrrrrrr...","{'score': 10, 'review': 'This is my favorite R...","{'score': 5, 'review': 'This game ruined Resid...","{'score': 9, 'review': 'If you have played the...","{'score': 10, 'review': 'the hard work and res...","{'score': 10, 'review': 'When i played this ga...","{'score': 10, 'review': 'Resident Evil 4 is wi...","{'score': 9, 'r